# Thüringen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Thüringen is `DEG`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [ ]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from glob import glob
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict, Tuple
from datetime import datetime as dt
from dateparser import parse
import warnings
from io import BytesIO

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [ ]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Thüringen').input_path
BASE

## Parse metadata

Pegel metadata can be read quite easily. Only the separator is important as we have whitespaces in `'Gewässer'`

In [ ]:
with Bundesland('Thüringen') as th:
    metadata = pd.read_csv(os.path.join(th.input_path, 'Pegel_Metadaten.txt'), sep='\t')
metadata['unit_q'] = 'm³/s'
metadata['unit_w'] = 'cm'
metadata

In [ ]:
# The id_column is Pegelnr.
id_column = 'Pegelnr.'

## Load Data

Loading data is a bit more complicated. There is an extra header, but that is not important. Only the 'Einhait' contains important information, but that has been added manually to the metadata.

In [ ]:
def extract_file(nr: Union[str, int], variable: str, base_path: str) -> Tuple[Dict[str, str], pd.DataFrame]:
    # always use str ids
    nr = str(nr)
    # merge the nr
    if '.' in nr:
        nr = nr.replace('.', '')

    # build the filename
    fname = f'{variable.lower()}{nr}.txt'

    # build the path
    path = os.path.join(base_path, variable.lower(), fname)

    # return empty dataframe if data does not exist
    if not os.path.exists(path):
        return pd.DataFrame(columns=['date', variable.lower(), 'flag'])
    
    # otherwise read
    df = pd.read_csv(path, skiprows=21, header=None, sep=" ", parse_dates=[0], dayfirst=True, na_values=['Luecke', 'LUECKE'])
    df.columns = ['date', 'hour', variable.lower(), 'comment']
    
    # check if there are comments
    if not df.comment.isna().all():
        print(df.comment)
    
    # build the flag column
    df['flag'] = [np.isnan(c) for c in df.comment]
    df.drop(['hour', 'comment'], axis=1, inplace=True)
    return df

extract_file('42012.0', 'w', BASE)

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [ ]:
with Bundesland('Thüringen') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    # go for each    
    for _id in tqdm(metadata[id_column].values):
        provider_id = str(_id)
        # extract the two files
        q_df = extract_file(provider_id, 'q', bl.input_path)
        w_df = extract_file(provider_id, 'w', bl.input_path)

        # save
        bl.save_timeseries(q_df, provider_id)
        bl.save_timeseries(w_df, provider_id)
